# Predict Sale Price

## Objectives

## Objectives

*   Fit and evaluate a regression model to predict the `SalePrice` of a house.

## Inputs

* outputs/datasets/collection/HeritageHousing.csv
* instructions on which variables to use for data cleaning and feature engineering. They are found on its respectives notebooks.

## Outputs

* Train set (features and target)
* Test set (features and target)
* Data cleaning and Feature Engineering pipeline
* Modeling pipeline
* Features importance plot


---

# Change working directory

* We are assuming you will store the notebooks in a sub folder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing-Issues-P5/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/Heritage-Housing-Issues-P5'

## Load Data

In [4]:
import numpy as np
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/HeritageHousing.csv")
    )

print(df.shape)
df.head(3)

(1460, 24)


,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,...,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,SalePrice
0,856,854.0,3.0,No,706,GLQ,150,0.0,548,RFn,...,65.0,196.0,61,5,7,856,0.0,2003,2003,208500
1,1262,0.0,3.0,Gd,978,ALQ,284,NaN,460,RFn,...,80.0,0.0,0,8,6,1262,NaN,1976,1976,181500
2,920,866.0,3.0,Mn,486,GLQ,434,0.0,608,RFn,...,68.0,162.0,42,5,7,920,NaN,2001,2002,223500


---

## ML Pipeline with all Data

### Cleaning and Feature Engineering

In [5]:
import seaborn as sns
sns.set(style="whitegrid")

### Data Cleaning
from sklearn.pipeline import Pipeline
from feature_engine.imputation import CategoricalImputer
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import ArbitraryNumberImputer


### Feature Engineering
from feature_engine.creation import MathFeatures
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine.encoding import OrdinalEncoder
from feature_engine.outliers import OutlierTrimmer
from feature_engine import transformation as vt

def PipelineDataCleaningAndFeatureEngineering():
  pipeline_base = Pipeline([
    ### Data Cleaning
    ("ArbitraryNumberImputer",ArbitraryNumberImputer(arbitrary_number=0, 
                                                variables=['2ndFlrSF', 'EnclosedPorch', 'MasVnrArea', 'WoodDeckSF'])),

    ("CategoricalImputer",CategoricalImputer(imputation_method='missing',fill_value='Unf',
                                                variables=['GarageFinish', 'BsmtFinType1'])),

    ("MeanMedianImputer",MeanMedianImputer(imputation_method='median', 
                                                variables=['LotFrontage', 'BedroomAbvGr'])),  

    ("YearFiller",df['GarageYrBlt'].fillna(df['YearBuilt'])),   

    ### Feature Engineering
    ("MathFeatures",MathFeatures(variables=['1stFlrSF','2ndFlrSF','BsmtFinSF1','BsmtUnfSF','EnclosedPorch',
        'GarageArea','GrLivArea','LotArea','MasVnrArea','OpenPorchSF','TotalBsmtSF','WoodDeckSF'],
        func=['sum'], new_variables_names=['TotalArea'])),

    ("OrdinalCategoricalEncoder",OrdinalEncoder(encoding_method='arbitrary', 
                                                variables = ['BsmtExposure', 'BsmtFinType1', 'GarageFinish', 'KitchenQual'])),

    ("LogTransformer",vt.LogTransformer(variables = ['1stFlrSF', 'GrLivArea', 'TotalArea', 'LotArea', 'LotFrontage', ])),

    ("PowerrTransformer",vt.PowerTransformer(variables = ['2ndFlrSF','BsmtFinSF1','BsmtUnfSF','GarageArea','MasVnrArea',
                                                        'OpenPorchSF','TotalBsmtSF','WoodDeckSF'])),

    ("OutlierTrimmer",OutlierTrimmer(capping_method='iqr', tail='both', fold=1.5, 
                                    variables=['SalePrice', 'GrLivArea', 'TotalArea'])),                                                    

    ("SmartCorrelatedSelection",SmartCorrelatedSelection(variables=None, method="spearman", 
                                                        threshold=0.6, selection_method="variance")),
       
  ])

  return pipeline_base

PipelineDataCleaningAndFeatureEngineering()

Pipeline(steps=[('ArbitraryNumberImputer',
                 ArbitraryNumberImputer(arbitrary_number=0,
                                        variables=['2ndFlrSF', 'EnclosedPorch',
                                                   'MasVnrArea',
                                                   'WoodDeckSF'])),
                ('CategoricalImputer',
                 CategoricalImputer(fill_value='Unf',
                                    variables=['GarageFinish',
                                               'BsmtFinType1'])),
                ('MeanMedianImputer',
                 MeanMedianImputer(variables=['LotFrontage', 'BedroomAbvGr'])),
                ('YearFiller',
                 0...
                 PowerTransformer(variables=['2ndFlrSF', 'BsmtFinSF1',
                                             'BsmtUnfSF', 'GarageArea',
                                             'MasVnrArea', 'OpenPorchSF',
                                             'TotalBsmtSF', 'WoodDeckSF'])),
                ('OutlierTrimmer',
                 OutlierTrimmer(capping_method='iqr', fold=1.5, tail='both',
                                variables=['SalePrice', 'GrLivArea',
                                           'TotalArea'])),
                ('SmartCorrelatedSelection',
                 SmartCorrelatedSelection(method='spearman',
                                          selection_method='variance',
                                          threshold=0.6))])

---

NOTE

* You may add how many sections you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section for "Conclusions and Next Steps" and state your conclusions and next steps.

In [6]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)


IndentationError: expected an indented block (2852421808.py, line 5)